<a href="https://colab.research.google.com/github/josegabrielguerrero26/CSE450-machinelearning/blob/main/project3/models_team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam

# Read data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

# Handle missing values
bikes['hr'] = bikes['hr'].fillna(bikes['hr'].shift(1) + 1)

# Create total_users feature
bikes["total_users"] = bikes["casual"] + bikes["registered"]

# Split features and target
y = bikes['total_users']
X = bikes.drop(columns=['casual', 'registered', 'total_users', 'dteday'])

# Normalize data
norm = MinMaxScaler().fit(X)
X_norm = norm.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

# Define model
model = Sequential([
    Dense(512, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')
])

# Compile model
optimizer = Adam(learning_rate=0.002)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['mse'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=30)

# Model training
history = model.fit(X_train, y_train, epochs=200, validation_split=0.35, batch_size=20,
                    callbacks=[early_stop], shuffle=True, verbose=1)

# Evaluate model
predictions = model.predict(X_test)
result = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print(f'MSE: {result:.3f}, R2: {r2:.3f}')

# Save predictions
bikesTest = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
bikesTest = bikesTest.drop(columns=['dteday'])

bikesTest_norm = norm.transform(bikesTest)
holdout_predictions = model.predict(bikesTest_norm)
rounded_predictions = np.round(holdout_predictions)
np.savetxt("team1-module4-predictions.csv", rounded_predictions, delimiter=",", header="predictions")


Epoch 1/200
2560/2560 [==============================] - 17s 6ms/step - loss: 64074.1406 - mse: 64074.1406 - val_loss: 49151.7695 - val_mse: 49151.7695
Epoch 2/200
2560/2560 [==============================] - 8s 3ms/step - loss: 50046.6719 - mse: 50046.6719 - val_loss: 43444.4219 - val_mse: 43444.4219
Epoch 3/200
2560/2560 [==============================] - 9s 4ms/step - loss: 46137.6914 - mse: 46137.6914 - val_loss: 39545.7773 - val_mse: 39545.7773
Epoch 4/200
2560/2560 [==============================] - 10s 4ms/step - loss: 42217.7734 - mse: 42217.7734 - val_loss: 35495.3867 - val_mse: 35495.3867
Epoch 5/200
2560/2560 [==============================] - 10s 4ms/step - loss: 39954.1953 - mse: 39954.1953 - val_loss: 34813.0039 - val_mse: 34813.0000
Epoch 6/200
2560/2560 [==============================] - 10s 4ms/step - loss: 38267.2773 - mse: 38267.2773 - val_loss: 36251.1875 - val_mse: 36251.1875
Epoch 7/200
2560/2560 [==============================] - 9s 3ms/step - loss: 37070.5508 - 

In [ ]:
from numpy.random import randint
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam

# Read data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

# Handle missing values
for row in bikes[bikes['hr'].isnull()].index.values:
    bikes.loc[row,'hr'] = bikes.loc[row-1,'hr']+1

# Create total_users feature
bikes["total_users"] = bikes["casual"] + bikes["registered"]

# Split features and target
y = bikes['total_users']
X = bikes.drop(columns=['casual', 'registered', 'total_users', 'dteday'])

# Normalize data
norm = MinMaxScaler().fit(X)
X_norm = norm.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

# Define model
model = Sequential([
    Dense(128, input_dim=len(X_train[1]), activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['mse'])

# Early stopping
early_stop = EarlyStopping(monitor='val_mse', patience=30)

# Model training
history = model.fit(X_train, y_train, epochs=100, validation_split=0.35, batch_size=20,
                    callbacks=[early_stop], shuffle=False, verbose=1)

# Evaluate model
predictions = model.predict(X_test)
result = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print(f'MSE: {result:.3f}, R2: {r2:.3f}')

# Save predictions
bikesTest = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
bikesTest = bikesTest.drop(columns=['dteday'])
bikesTest_norm = norm.transform(bikesTest)
holdout_predictions = model.predict(bikesTest_norm)
rounded_predictions = np.round(holdout_predictions)
np.savetxt("team1-module4-predictions.csv", rounded_predictions, delimiter=",", header="predictions")


Epoch 1/100
2560/2560 [==============================] - 10s 3ms/step - loss: 84560.7812 - mse: 84560.7812 - val_loss: 65912.5703 - val_mse: 65912.5703
Epoch 2/100
2560/2560 [==============================] - 10s 4ms/step - loss: 63098.7969 - mse: 63098.7969 - val_loss: 52416.6680 - val_mse: 52416.6719
Epoch 3/100
2560/2560 [==============================] - 8s 3ms/step - loss: 55594.2773 - mse: 55594.2773 - val_loss: 48142.3789 - val_mse: 48142.3789
Epoch 4/100
2560/2560 [==============================] - 8s 3ms/step - loss: 51877.3711 - mse: 51877.3711 - val_loss: 46380.6680 - val_mse: 46380.6680
Epoch 5/100
2560/2560 [==============================] - 9s 3ms/step - loss: 49821.0391 - mse: 49821.0391 - val_loss: 45149.8398 - val_mse: 45149.8398
Epoch 6/100
2560/2560 [==============================] - 7s 3ms/step - loss: 48402.6016 - mse: 48402.6016 - val_loss: 43729.2773 - val_mse: 43729.2773
Epoch 7/100
2560/2560 [==============================] - 9s 3ms/step - loss: 47414.1445 - ms

In [ ]:
from numpy.random import randint
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam

# Read data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

# Handle missing values
bikes['hr'] = bikes['hr'].fillna(bikes['hr'].shift(1) + 1)

# Create total_users feature
bikes["total_users"] = bikes["casual"] + bikes["registered"]

# Split features and target
y = bikes['total_users']
X = bikes.drop(columns=['casual', 'registered', 'total_users', 'dteday'])

# Normalize data
norm = MinMaxScaler().fit(X)
X_norm = norm.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

# Define model
model = Sequential([
    Dense(128, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile model
optimizer = Adam(learning_rate=0.0005)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['mse'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=30)

# Model training
history = model.fit(X_train, y_train, epochs=100, validation_split=0.35, batch_size=20,
                    callbacks=[early_stop], shuffle=True, verbose=1)

# Evaluate model
predictions = model.predict(X_test)
result = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print(f'MSE: {result:.3f}, R2: {r2:.3f}')

# Save predictions
bikesTest = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
bikesTest = bikesTest.drop(columns=['dteday'])
bikesTest_norm = norm.transform(bikesTest)
holdout_predictions = model.predict(bikesTest_norm)
rounded_predictions = np.round(holdout_predictions)
np.savetxt("team1-module4-predictions.csv", rounded_predictions, delimiter=",", header="predictions")

Epoch 1/100
2560/2560 [==============================] - 9s 3ms/step - loss: 92454.2969 - mse: 92454.2969 - val_loss: 74368.6719 - val_mse: 74368.6719
Epoch 2/100
2560/2560 [==============================] - 8s 3ms/step - loss: 72490.7891 - mse: 72490.7891 - val_loss: 63528.7266 - val_mse: 63528.7266
Epoch 3/100
2560/2560 [==============================] - 8s 3ms/step - loss: 63979.2109 - mse: 63979.2109 - val_loss: 56303.7539 - val_mse: 56303.7539
Epoch 4/100
2560/2560 [==============================] - 8s 3ms/step - loss: 58494.8477 - mse: 58494.8477 - val_loss: 51856.0156 - val_mse: 51856.0156
Epoch 5/100
2560/2560 [==============================] - 8s 3ms/step - loss: 54982.1875 - mse: 54982.1875 - val_loss: 49624.3398 - val_mse: 49624.3398
Epoch 6/100
2560/2560 [==============================] - 9s 3ms/step - loss: 52742.8711 - mse: 52742.8711 - val_loss: 48568.7070 - val_mse: 48568.7070
Epoch 7/100
2560/2560 [==============================] - 9s 3ms/step - loss: 51337.9766 - mse:

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dropout, Dense, BatchNormalization, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import Adam

# Read data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

# Handle missing values
bikes['hr'] = bikes['hr'].fillna(bikes['hr'].shift(1) + 1)

# Create total_users feature
bikes["total_users"] = bikes["casual"] + bikes["registered"]

# Split features and target
y = bikes['total_users']
X = bikes.drop(columns=['casual', 'registered', 'total_users', 'dteday'])

# Normalize data
norm = MinMaxScaler().fit(X)
X_norm = norm.transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

# Define model
model = Sequential([
    Dense(512, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')
])

# Compile model
optimizer = Adam(learning_rate=0.0001)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['mse'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=30)

# Model training
history = model.fit(X_train, y_train, epochs=200, validation_split=0.35, batch_size=20,
                    callbacks=[early_stop], shuffle=True, verbose=1)

# Evaluate model
predictions = model.predict(X_test)
result = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)
print(f'MSE: {result:.3f}, R2: {r2:.3f}')

# Save predictions
bikesTest = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes_december.csv')
bikesTest = bikesTest.drop(columns=['dteday'])

bikesTest_norm = norm.transform(bikesTest)
holdout_predictions = model.predict(bikesTest_norm)
rounded_predictions = np.round(holdout_predictions)
np.savetxt("team1-module4-predictions.csv", rounded_predictions, delimiter=",", header="predictions")


Epoch 1/200
2560/2560 [==============================] - 16s 6ms/step - loss: 95684.5469 - mse: 95684.5469 - val_loss: 74672.7188 - val_mse: 74672.7188
Epoch 2/200
2560/2560 [==============================] - 15s 6ms/step - loss: 72595.8438 - mse: 72595.8359 - val_loss: 67113.4297 - val_mse: 67113.4297
Epoch 3/200
2560/2560 [==============================] - 15s 6ms/step - loss: 64323.5352 - mse: 64323.5352 - val_loss: 58003.4648 - val_mse: 58003.4648
Epoch 4/200
2560/2560 [==============================] - 15s 6ms/step - loss: 57863.9180 - mse: 57863.9180 - val_loss: 53080.5859 - val_mse: 53080.5859
Epoch 5/200
2560/2560 [==============================] - 18s 7ms/step - loss: 54371.4766 - mse: 54371.4766 - val_loss: 50050.2891 - val_mse: 50050.2891
Epoch 6/200
2560/2560 [==============================] - 15s 6ms/step - loss: 52189.3633 - mse: 52189.3633 - val_loss: 48427.7305 - val_mse: 48427.7305
Epoch 7/200
2560/2560 [==============================] - 15s 6ms/step - loss: 50405.8594

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Read data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

# Handle missing values
bikes['hr'] = bikes['hr'].fillna(method='ffill')

# Create total_users feature
bikes["total_users"] = bikes["casual"] + bikes["registered"]

# Split features and target
y = bikes['total_users']
X = bikes.drop(columns=['casual', 'registered', 'total_users', 'dteday'])

# Normalize data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Define the model
model = Sequential([
    Dense(256, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.00.05)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['mse'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Model training
history = model.fit(X_train, y_train, epochs=200, validation_split=0.2, batch_size=20, callbacks=[early_stop], verbose=1)

# Evaluate model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse:.3f}, R2: {r2:.3f}')

# Save predictions
bikesTest = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
bikesTest = bikesTest.drop(columns=['dteday'])
bikesTest_scaled = scaler.transform(bikesTest)
holdout_predictions = model.predict(bikesTest_scaled)
rounded_predictions = np.round(holdout_predictions)
np.savetxt("best_model_predictions.csv", rounded_predictions, delimiter=",", header="predictions")

Epoch 1/200
3150/3150 [==============================] - 17s 5ms/step - loss: 72751.1484 - mse: 72751.1484 - val_loss: 46789.9648 - val_mse: 46789.9648
Epoch 2/200
3150/3150 [==============================] - 13s 4ms/step - loss: 53267.4492 - mse: 53267.4492 - val_loss: 40871.7930 - val_mse: 40871.7930
Epoch 3/200
3150/3150 [==============================] - 13s 4ms/step - loss: 49709.5742 - mse: 49709.5742 - val_loss: 35479.0195 - val_mse: 35479.0195
Epoch 4/200
3150/3150 [==============================] - 13s 4ms/step - loss: 47070.6211 - mse: 47070.6211 - val_loss: 34560.5977 - val_mse: 34560.5977
Epoch 5/200
3150/3150 [==============================] - 13s 4ms/step - loss: 46032.6719 - mse: 46032.6719 - val_loss: 32639.6055 - val_mse: 32639.6055
Epoch 6/200
3150/3150 [==============================] - 13s 4ms/step - loss: 44986.6562 - mse: 44986.6562 - val_loss: 32781.4961 - val_mse: 32781.4961
Epoch 7/200
3150/3150 [==============================] - 13s 4ms/step - loss: 44542.8281

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Read data
bikes = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bikes.csv')

# Handle missing values
bikes['hr'] = bikes['hr'].fillna(method='ffill')

# Create total_users feature
bikes["total_users"] = bikes["casual"] + bikes["registered"]

# Split features and target
y = bikes['total_users']
X = bikes.drop(columns=['casual', 'registered', 'total_users', 'dteday'])

# Normalize data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Define the model
model = Sequential([
    Dense(512, input_dim=X_train.shape[1], activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0005)
model.compile(loss="mean_squared_error", optimizer=optimizer, metrics=['mse'])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)

# Model training
history = model.fit(X_train, y_train, epochs=100, validation_split=0.35, batch_size=20, callbacks=[early_stop], verbose=1)

# Evaluate model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse:.3f}, R2: {r2:.3f}')

# Save predictions
bikesTest = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/biking_holdout_test_mini.csv')
bikesTest = bikesTest.drop(columns=['dteday'])
bikesTest_scaled = scaler.transform(bikesTest)
holdout_predictions = model.predict(bikesTest_scaled)
rounded_predictions = np.round(holdout_predictions)
np.savetxt("team1-module4-predictions.csv", rounded_predictions, delimiter=",", header="predictions")


Epoch 1/100
2560/2560 [==============================] - 21s 7ms/step - loss: 74730.0312 - mse: 74730.0312 - val_loss: 43248.1328 - val_mse: 43248.1328
Epoch 2/100
2560/2560 [==============================] - 18s 7ms/step - loss: 54108.9102 - mse: 54108.9102 - val_loss: 37496.6641 - val_mse: 37496.6641
Epoch 3/100
2560/2560 [==============================] - 18s 7ms/step - loss: 50487.9727 - mse: 50487.9727 - val_loss: 35632.9062 - val_mse: 35632.9023
Epoch 4/100
2560/2560 [==============================] - 18s 7ms/step - loss: 47141.8906 - mse: 47141.8906 - val_loss: 36880.1289 - val_mse: 36880.1289
Epoch 5/100
2560/2560 [==============================] - 19s 7ms/step - loss: 45601.9414 - mse: 45601.9414 - val_loss: 32050.3398 - val_mse: 32050.3398
Epoch 6/100
2560/2560 [==============================] - 17s 7ms/step - loss: 44294.7422 - mse: 44294.7422 - val_loss: 32394.8125 - val_mse: 32394.8125
Epoch 7/100
2560/2560 [==============================] - 18s 7ms/step - loss: 43405.3984